In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model

## Import the data into Pandas data frame

In [ ]:
data = pd.read_csv('Telco-Customer-Churn.csv')

In [ ]:
data.head()

In [ ]:
data.info()

## Data Cleansing
Senior Citizen column has two values, Yes and No but here it’s 1 and 0 and column is a int64 type. Let’s fix that. first we need to replace 1s and 0s with Yes and No:

In [ ]:
data.SeniorCitizen.replace([0, 1], ["No", "Yes"], inplace= True)

TotalCharges column should be numerical.

In [ ]:
data.TotalCharges = data.TotalCharges.astype(float)

In [ ]:
for charge in data.TotalCharges:
    try:
        charge = float(charge)
    except:
        print("charge is: %s" % charge)

But when we try to convert this column in to float data type it raises an error. That’s because this column has some blank cells. Let’s go a bit deep in to this rabbit hole

Checking in this way it shows that 11 rows has a space as TotalCharge. What do we do for this? Let’s check what are the MonthlyCharge and Tenure when TotalCharge is a space/blank value

In [ ]:
for i in range(len(data)):
  if data.TotalCharges[i] == " ":
      print("Tenure is %s and Monthly charges are %s" % (data.tenure[i], data.MonthlyCharges[i]))

So, it’s obvious that total charges are blank when tenure is 0 in other words they are pretty new and short timed customers. Since their tenure is zero it’s quite logical to think that their total charges as zero. Let’s replace space in TotalCharges to 0 and convert the column to float data type.

In [ ]:
data.TotalCharges.replace([" "], ["0"], inplace= True)
data.TotalCharges = data.TotalCharges.astype(float)

## Lets Drop Irrelevant features

In [ ]:
data.drop("customerID", axis= 1, inplace= True)

In [ ]:
for col in data.dtypes[data.dtypes == object].index:
    print(col, data[col].unique())

Now we have 3 numerical columns and rest are categorical. In order to normalize the numerical columns let’s divide the values of each column by the max value of the relevant column. This will make sure all the values are between 0 - 1 thus, helping to reduce those values giving unwanted weighing on final model.

Since our labels will be Churn and it has Yes, No values, let’s replace Yes with one and No with zero so our model will be a binary classification model.

In [ ]:
data.Churn.replace(["Yes", "No"], [1, 0], inplace= True)

Next, we should encode all the categorical data in to one-hot encoding. Pandas have a quite easy way to do that:

In [ ]:
data = pd.get_dummies(data)

## Select X and Y features

In [ ]:
X = data.drop("Churn", axis= 1)
y = data.Churn

Inputs to the model will be X and it has all the columns except Churn which is our label thus, assign it to y. To train and test our model we should split our dataset. This split will create 20% of data for testing and 80% for training.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 1234)

In [ ]:
model = Sequential()
model.add(Dense(16, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, 
                    y_train, 
                    epochs=150, 
                    batch_size=10
                   validation_split=0.1)

In [ ]:
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend(['train'], loc='upper left')
plt.show()

## Evaluate the model

In [ ]:
model.evaluate(X_train, y_train)

In [ ]:
model.evaluate(X_test, y_test)